In [4]:
KAFKA_BOOTSTRAP = "ec2-3-8-99-245.eu-west-2.compute.amazonaws.com:9092"
TOPIC = "stocks_demo"
CSV_PATH = r"C:\Users\Administrator\OneDrive - KOROGLU AGATHA\DE\kafka_streaming\kafka_project\indexProcessed.csv"
SEND_INTERVAL_SEC = 1.0   # 1 message per second (change to taste)


In [ ]:
import pandas as pd, json, time, random
from kafka import KafkaProducer

# Load once, sample rows forever
df = pd.read_csv(CSV_PATH)

producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BOOTSTRAP],
    acks="all",
    linger_ms=0,  # send immediately (lowest latency)
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

try:
    i = 0
    while True:
        rec = df.iloc[random.randrange(len(df))].to_dict()
        # Optional latency measurement:
        # rec["_sent_ts_ns"] = time.time_ns()
        producer.send(TOPIC, rec)
        # producer.flush()  # not needed with linger_ms=0; uncomment if you want hard flush per msg
        i += 1
        if i % 100 == 0:
            print(f"sent {i} messages…")
        time.sleep(SEND_INTERVAL_SEC)
finally:
    producer.flush()
    producer.close()


sent 100 messages…
sent 200 messages…
sent 300 messages…
sent 400 messages…
sent 500 messages…
sent 600 messages…
sent 700 messages…
sent 800 messages…
sent 900 messages…
sent 1000 messages…
sent 1100 messages…
sent 1200 messages…
sent 1300 messages…
sent 1400 messages…
sent 1500 messages…
sent 1600 messages…
sent 1700 messages…
sent 1800 messages…
sent 1900 messages…
sent 2000 messages…
sent 2100 messages…
sent 2200 messages…
sent 2300 messages…
sent 2400 messages…
sent 2500 messages…
sent 2600 messages…
sent 2700 messages…
sent 2800 messages…
sent 2900 messages…
sent 3000 messages…
sent 3100 messages…
sent 3200 messages…
sent 3300 messages…
sent 3400 messages…
sent 3500 messages…
sent 3600 messages…
sent 3700 messages…
sent 3800 messages…
sent 3900 messages…
sent 4000 messages…
sent 4100 messages…
sent 4200 messages…
sent 4300 messages…
sent 4400 messages…
sent 4500 messages…
sent 4600 messages…
sent 4700 messages…
sent 4800 messages…
sent 4900 messages…
sent 5000 messages…
sent 5100